In [1]:
import os
from pathlib import Path

In [2]:
os.environ["DAGSTER_HOME"] = str(Path().resolve().parent / "work")

In [3]:
from data_orchestration import defs
from data_orchestration.opentecr.helpers import transform_compound_name

/home/moritz/Codebase/opentecr/data-orchestration/src/data_orchestration/opentecr/asset/bronze.py:29: ExperimentalWarning: Parameter `tags` of function `graph_asset` is experimental. It may break in future versions, even between dot releases. To mute warnings for experimental functionality, invoke warnings.filterwarnings("ignore", category=dagster.ExperimentalWarning) or use one of the other methods described at https://docs.python.org/3/library/warnings.html#describing-warning-filters.
  @graph_asset(
/home/moritz/.pyenv/versions/tecr-data/lib/python3.12/site-packages/dagster/_core/definitions/decorators/asset_decorator.py:767: ExperimentalWarning: Parameter `tags` of function `graph_asset` is experimental. It may break in future versions, even between dot releases. To mute warnings for experimental functionality, invoke warnings.filterwarnings("ignore", category=dagster.ExperimentalWarning) or use one of the other methods described at https://docs.python.org/3/library/warnings.html#d

In [4]:
reactions = defs.load_asset_value("opentecr_unique_reactions")

2024-08-02 00:25:00 +0200 - dagster - DEBUG - system - Loading file from: /home/moritz/Codebase/opentecr/data-orchestration/work/storage/opentecr_unique_reactions.arrow using PandasArrowIOManager...


In [5]:
reactions.loc[reactions["reaction"].str.contains(" - "), "reaction"].unique()

array([], dtype=object)

In [6]:
compounds = reactions["reaction"].str.split(
    "=",
    n=1,
    expand=True,
    regex=False,
).rename(columns={0: "reactants", 1: "products"})

In [7]:
compounds

,reactants,products
0,benzyl alcohol(aq) + NAD(aq),benzaldehyde(aq) + NADH(aq)
1,1-butanol(aq) + NAD(aq),butanal(aq) + NADH(aq)
2,cyclohexanol(aq) + NAD(aq),cyclohexanone(aq) + NADH(aq)
3,ethanol(aq) + NAD(aq),acetaldehyde(aq) + NADH(aq)
4,ethanol(aq) + AP-NAD(aq),acetaldehyde(aq) + AP-NADH(aq)
...,...,...
905,ATP(aq) + L-serine(aq) + tRNA-Ser(aq),AMP(aq) + pyrophosphate(aq) + L-seryl-tRNA-Se...
906,ATP(aq) + L-arginine(aq) + tRNA-Arg(aq),AMP(aq) + pyrophosphate(aq) + L-arginyl-tRNA-...
907,ATP(aq) + L-phenylalanine(aq) + tRNA-Phe(aq),AMP(aq) + pyrophosphate(aq) + L-phenylalanyl-...
908,ATP(aq) + L-histidine(aq) + tRNA-His(aq),AMP(aq) + pyrophosphate(aq) + L-histidyl-tRNA...


In [8]:
compounds["reactants"].map(
            lambda lhs: [
                transform_compound_name(name)
                for name in lhs.split(" + ")
            ], na_action="ignore"
        )

0                                  [benzyl alcohol, NAD]
1                                       [1-butanol, NAD]
2                                    [cyclohexanol, NAD]
3                                         [ethanol, NAD]
4                                      [ethanol, AP-NAD]
                             ...                        
905                            [ATP, L-serine, tRNA-Ser]
906                          [ATP, L-arginine, tRNA-Arg]
907                     [ATP, L-phenylalanine, tRNA-Phe]
908                         [ATP, L-histidine, tRNA-His]
909    [ATP, (deoxyribonucleotide)_{n}, (deoxyribonuc...
Name: reactants, Length: 910, dtype: object

In [9]:
compounds["products"].map(
            lambda rhs: [
                transform_compound_name(name)
                for name in rhs.split(" + ")
            ], na_action="ignore"
        )

0                                   [benzaldehyde, NADH]
1                                        [butanal, NADH]
2                                  [cyclohexanone, NADH]
3                                   [acetaldehyde, NADH]
4                                [acetaldehyde, AP-NADH]
                             ...                        
905               [AMP, pyrophosphate, L-seryl-tRNA-Ser]
906             [AMP, pyrophosphate, L-arginyl-tRNA-Arg]
907        [AMP, pyrophosphate, L-phenylalanyl-tRNA-Phe]
908            [AMP, pyrophosphate, L-histidyl-tRNA-His]
909    [AMP, pyrophosphate, (deoxyribonucleotide)_{m+n}]
Name: products, Length: 910, dtype: object

In [10]:
result = (
    reactions.loc[:, ["reaction_hash"]]
    .assign(
        reactant=compounds["reactants"].map(
            lambda lhs: [
                transform_compound_name(name)
                for name in lhs.split(" + ")
            ], na_action="ignore"
        ),
        product=compounds["products"].map(
            lambda rhs: [
                transform_compound_name(name)
                for name in rhs.split(" + ")
            ], na_action="ignore"
        ),
    )
)

In [11]:
result

,reaction_hash,reactant,product
0,eda7a93ff5f4e8f4f57f11b63f57ef4c37f9745f,"[benzyl alcohol, NAD]","[benzaldehyde, NADH]"
1,8dc493fc781054323abddd37a24c083c56bd29a6,"[1-butanol, NAD]","[butanal, NADH]"
2,a9d4e658adf5b52e923c33b5f8ad8a1ac72e9cf3,"[cyclohexanol, NAD]","[cyclohexanone, NADH]"
3,c0b83fb9adaacbb9f520d3a55713d1fe4c8232b5,"[ethanol, NAD]","[acetaldehyde, NADH]"
4,b654853e259e90400925dae3b4cff2bb170083d3,"[ethanol, AP-NAD]","[acetaldehyde, AP-NADH]"
...,...,...,...
905,9b68b7c142fe45b7dacd06c31d55202390d8ffc3,"[ATP, L-serine, tRNA-Ser]","[AMP, pyrophosphate, L-seryl-tRNA-Ser]"
906,936e793e574b3a2a9f1f243c4bc291980fd53ee3,"[ATP, L-arginine, tRNA-Arg]","[AMP, pyrophosphate, L-arginyl-tRNA-Arg]"
907,b1406c7559aedd7a9e8e9eb011baa4d9e839720d,"[ATP, L-phenylalanine, tRNA-Phe]","[AMP, pyrophosphate, L-phenylalanyl-tRNA-Phe]"
908,bc83f26880815041458dcc2091c4ec3a5a94259d,"[ATP, L-histidine, tRNA-His]","[AMP, pyrophosphate, L-histidyl-tRNA-His]"


In [12]:
result.melt(id_vars=["reaction_hash"], value_vars=["reactant", "product"], var_name="reaction_side", value_name="compounds").explode("compounds", ignore_index=True)

,reaction_hash,reaction_side,compounds
0,eda7a93ff5f4e8f4f57f11b63f57ef4c37f9745f,reactant,benzyl alcohol
1,eda7a93ff5f4e8f4f57f11b63f57ef4c37f9745f,reactant,NAD
2,8dc493fc781054323abddd37a24c083c56bd29a6,reactant,1-butanol
3,8dc493fc781054323abddd37a24c083c56bd29a6,reactant,NAD
4,a9d4e658adf5b52e923c33b5f8ad8a1ac72e9cf3,reactant,cyclohexanol
...,...,...,...
3430,bc83f26880815041458dcc2091c4ec3a5a94259d,product,pyrophosphate
3431,bc83f26880815041458dcc2091c4ec3a5a94259d,product,L-histidyl-tRNA-His
3432,97f000c788da00a79622d519ba2cbe10e6ecf176,product,AMP
3433,97f000c788da00a79622d519ba2cbe10e6ecf176,product,pyrophosphate
